# Data Preprocessing

In [1]:
import pandas as pd
import swifter
import pickle as pkl

%run "../shared.py"

pd.set_option('display.max_colwidth', 100)

In [2]:
target_dataset = 'tweets.csv'
raw_data_path = f'../dataset/raw/raw_{target_dataset}'
output_path = f'../dataset/processed/processed_{target_dataset}'

In [3]:
df = pd.read_csv(raw_data_path, delimiter='`')

print(df.shape)

(7502, 27)


In [4]:
df = df[['date','content']]

print(df.shape)
df

(7502, 2)


,date,content
0,2022-09-04 23:59:55+00:00,@soeyoto1 @msaid_didu Bukti kalau Pemerintahan Pak @jokowi tidak pro rakyat. Tapi pro oligarki. ...
1,2022-09-04 23:59:38+00:00,"@KompasTV Mentri esdm, trus patokanmu apa? Kantong pertamina udah jebol karna banyak dikorupsi, ..."
2,2022-09-04 23:59:09+00:00,@hnurwahid Ngaku masih subsidi tapi ada swasta yg jua lebih murah? mereka siapa yg subsidi? Pert...
3,2022-09-04 23:58:55+00:00,@Tan_Mar3M Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina ? 🤔
4,2022-09-04 23:57:45+00:00,Menteri BUMN Erick Thohir menegaskan persiapan Indonesia soal transisi energi baru terbarukan. S...
...,...,...
7497,2022-09-03 00:00:54+00:00,"175 Produk UMKM Binaan Pertamina Go Global, Ikut Festival Tong Tong di Belanda https://t.co/LTf7..."
7498,2022-09-03 00:00:36+00:00,"PT Pertamina Patra Niaga, Sub Holding Commercial dan Trading PT Pertamina (Persero) menyebutkan ..."
7499,2022-09-03 00:00:02+00:00,Pemerintah Provinsi Papua Barat sedang mengupayakan dengan mengajak Pertamina agar masalah antre...
7500,2022-09-03 00:00:01+00:00,"terhitung mulai 1 September 2020 pukul 00.00 waktu setempat, Pertamina secara resmi melakukan pe..."


# Section 1: Data cleaning

In [5]:
list_normalize_targets, list_normalize_replacements = load_normalization_list()
list_stopwords = load_stopwords_list()

Normalization list: 1043
Indonesian stopwords: 540
English stopwords: 179
Custom stopwords: 17
Total stopwords: 736


## Section 1.1: Case folding, data cleaning and tokenization data

In [6]:
def combine_tokens(tokens):
	return ' '.join(tokens)

df['text_clean'] = df['content'].swifter.apply(clean_tweet)
df['case_folding'] = df['text_clean'].swifter.apply(case_folding)
df['tokens'] = df['case_folding'].swifter.apply(tokenize)
df['review'] = df['tokens'].swifter.apply(combine_tokens)

print(df.shape)
df

Pandas Apply:   0%|          | 0/7502 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7502 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7502 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7502 [00:00<?, ?it/s]

(7502, 6)


,date,content,text_clean,case_folding,tokens,review
0,2022-09-04 23:59:55+00:00,@soeyoto1 @msaid_didu Bukti kalau Pemerintahan Pak @jokowi tidak pro rakyat. Tapi pro oligarki. ...,Bukti kalau Pemerintahan Pak tidak pro rakyat Tapi pro oligarki Vivo BBMnya lebih murah dari Pem...,bukti kalau pemerintahan pak tidak pro rakyat tapi pro oligarki vivo bbmnya lebih murah dari pem...,"[bukti, kalau, pemerintahan, pak, tidak, pro, rakyat, tapi, pro, oligarki, vivo, bbmnya, lebih, ...",bukti kalau pemerintahan pak tidak pro rakyat tapi pro oligarki vivo bbmnya lebih murah dari pem...
1,2022-09-04 23:59:38+00:00,"@KompasTV Mentri esdm, trus patokanmu apa? Kantong pertamina udah jebol karna banyak dikorupsi, ...",Mentri esdm trus patokanmu apa Kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...,mentri esdm trus patokanmu apa kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...,"[mentri, esdm, trus, patokanmu, apa, kantong, pertamina, udah, jebol, karna, banyak, dikorupsi, ...",mentri esdm trus patokanmu apa kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...
2,2022-09-04 23:59:09+00:00,@hnurwahid Ngaku masih subsidi tapi ada swasta yg jua lebih murah? mereka siapa yg subsidi? Pert...,Ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi Pertamina dipimpin tuka...,ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi pertamina dipimpin tuka...,"[ngaku, masih, subsidi, tapi, ada, swasta, jua, lebih, murah, mereka, siapa, subsidi, pertamina,...",ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi pertamina dipimpin tuka...
3,2022-09-04 23:58:55+00:00,@Tan_Mar3M Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina ? 🤔,Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina,kalau vivo bisaberani bertahan dengan harga lama bagaimana pula nasib pertamina,"[kalau, vivo, bisaberani, bertahan, dengan, harga, lama, bagaimana, pula, nasib, pertamina]",kalau vivo bisaberani bertahan dengan harga lama bagaimana pula nasib pertamina
4,2022-09-04 23:57:45+00:00,Menteri BUMN Erick Thohir menegaskan persiapan Indonesia soal transisi energi baru terbarukan. S...,Menteri BUMNErick Thohirmenegaskan persiapan Indonesia soal transisi energi baru terbarukan Sala...,menteri bumnerick thohirmenegaskan persiapan indonesia soal transisi energi baru terbarukan sala...,"[menteri, bumnerick, thohirmenegaskan, persiapan, indonesia, soal, transisi, energi, baru, terba...",menteri bumnerick thohirmenegaskan persiapan indonesia soal transisi energi baru terbarukan sala...
...,...,...,...,...,...,...
7497,2022-09-03 00:00:54+00:00,"175 Produk UMKM Binaan Pertamina Go Global, Ikut Festival Tong Tong di Belanda https://t.co/LTf7...",Produk UMKM Binaan Pertamina Global Ikut Festival Tong Tong Belanda,produk umkm binaan pertamina global ikut festival tong tong belanda,"[produk, umkm, binaan, pertamina, global, ikut, festival, tong, tong, belanda]",produk umkm binaan pertamina global ikut festival tong tong belanda
7498,2022-09-03 00:00:36+00:00,"PT Pertamina Patra Niaga, Sub Holding Commercial dan Trading PT Pertamina (Persero) menyebutkan ...",Pertamina Patra Niaga Sub Holding Commercial dan Trading Pertamina Persero menyebutkan telah mel...,pertamina patra niaga sub holding commercial dan trading pertamina persero menyebutkan telah mel...,"[pertamina, patra, niaga, sub, holding, commercial, dan, trading, pertamina, persero, menyebutka...",pertamina patra niaga sub holding commercial dan trading pertamina persero menyebutkan telah mel...
7499,2022-09-03 00:00:02+00:00,Pemerintah Provinsi Papua Barat sedang mengupayakan dengan mengajak Pertamina agar masalah antre...,Pemerintah Provinsi Papua Barat sedang mengupayakan dengan mengajak Pertamina agar masalah antre...,pemerintah provinsi papua barat sedang mengupayakan dengan mengajak pertamina agar masalah antre...,"[pemerintah, provinsi, papua, barat, sedang, mengupayaka

## Section 1.2: Normalization and Stopwords removal

In [7]:
df['tokens'] = df['tokens'].swifter.apply(normalize, args=(list_normalize_targets, list_normalize_replacements))
df['tokens'] = df['tokens'].swifter.apply(remove_stopwords, args=(list_stopwords,))
df['review'] = df['tokens'].swifter.apply(combine_tokens)

print(df.shape)
df

Pandas Apply:   0%|          | 0/7502 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7502 [00:00<?, ?it/s]

(7502, 6)


,date,content,text_clean,case_folding,tokens,review
0,2022-09-04 23:59:55+00:00,@soeyoto1 @msaid_didu Bukti kalau Pemerintahan Pak @jokowi tidak pro rakyat. Tapi pro oligarki. ...,Bukti kalau Pemerintahan Pak tidak pro rakyat Tapi pro oligarki Vivo BBMnya lebih murah dari Pem...,bukti kalau pemerintahan pak tidak pro rakyat tapi pro oligarki vivo bbmnya lebih murah dari pem...,"[bukti, kalau, pemerintahan, tidak, pro, rakyat, pro, oligarki, vivo, bbmnya, lebih, murah, dari...",bukti kalau pemerintahan tidak pro rakyat pro oligarki vivo bbmnya lebih murah dari pemerintah n...
1,2022-09-04 23:59:38+00:00,"@KompasTV Mentri esdm, trus patokanmu apa? Kantong pertamina udah jebol karna banyak dikorupsi, ...",Mentri esdm trus patokanmu apa Kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...,mentri esdm trus patokanmu apa kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...,"[mentri, esdm, trus, patokanmu, apa, kantong, pertamina, udah, jebol, karna, banyak, dikorupsi, ...",mentri esdm trus patokanmu apa kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...
2,2022-09-04 23:59:09+00:00,@hnurwahid Ngaku masih subsidi tapi ada swasta yg jua lebih murah? mereka siapa yg subsidi? Pert...,Ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi Pertamina dipimpin tuka...,ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi pertamina dipimpin tuka...,"[ngaku, subsidi, ada, swasta, jua, lebih, murah, siapa, subsidi, pertamina, dipimpin, tukang, ma...",ngaku subsidi ada swasta jua lebih murah siapa subsidi pertamina dipimpin tukang maki jadi begin...
3,2022-09-04 23:58:55+00:00,@Tan_Mar3M Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina ? 🤔,Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina,kalau vivo bisaberani bertahan dengan harga lama bagaimana pula nasib pertamina,"[kalau, vivo, bisaberani, bertahan, harga, lama, nasib, pertamina]",kalau vivo bisaberani bertahan harga lama nasib pertamina
4,2022-09-04 23:57:45+00:00,Menteri BUMN Erick Thohir menegaskan persiapan Indonesia soal transisi energi baru terbarukan. S...,Menteri BUMNErick Thohirmenegaskan persiapan Indonesia soal transisi energi baru terbarukan Sala...,menteri bumnerick thohirmenegaskan persiapan indonesia soal transisi energi baru terbarukan sala...,"[menteri, bumnerick, thohirmenegaskan, persiapan, indonesia, soal, transisi, energi, terbarukan,...",menteri bumnerick thohirmenegaskan persiapan indonesia soal transisi energi terbarukan salah sat...
...,...,...,...,...,...,...
7497,2022-09-03 00:00:54+00:00,"175 Produk UMKM Binaan Pertamina Go Global, Ikut Festival Tong Tong di Belanda https://t.co/LTf7...",Produk UMKM Binaan Pertamina Global Ikut Festival Tong Tong Belanda,produk umkm binaan pertamina global ikut festival tong tong belanda,"[produk, umkm, binaan, pertamina, global, ikut, festival, tong, tong, belanda]",produk umkm binaan pertamina global ikut festival tong tong belanda
7498,2022-09-03 00:00:36+00:00,"PT Pertamina Patra Niaga, Sub Holding Commercial dan Trading PT Pertamina (Persero) menyebutkan ...",Pertamina Patra Niaga Sub Holding Commercial dan Trading Pertamina Persero menyebutkan telah mel...,pertamina patra niaga sub holding commercial dan trading pertamina persero menyebutkan telah mel...,"[pertamina, patra, niaga, sub, holding, commercial, trading, pertamina, persero, penyesuaian, ha...",pertamina patra niaga sub holding commercial trading pertamina persero penyesuaian harga bbm non...
7499,2022-09-03 00:00:02+00:00,Pemerintah Provinsi Papua Barat sedang mengupayakan dengan mengajak Pertamina agar masalah antre...,Pemerintah Provinsi Papua Barat sedang mengupayakan dengan mengajak Pertamina agar masalah antre...,pemerintah provinsi papua barat sedang mengupayakan dengan mengajak pertamina agar masalah antre...,"[pemerintah, provinsi, papua, barat, mengupayakan, mengajak, pertamina, agar, masalah, antrean, ...",pe

## Section 1.3: Remove Data

In [8]:
from collections import Counter

def remove_data_with_rare_word(df, key_name='tokens', threshold=2):
	print(f"Before: {df.shape}")
	token_counts = Counter([token for tokens in df[key_name] for token in tokens])
	filtered_words = [token for token, count in token_counts.items() if count >= threshold]
	df = df[df[key_name].apply(lambda x: any(token in filtered_words for token in x))]
	df = df.reset_index(drop=True)
	print(f"After: {df.shape}")
	return df

def remove_data(df, key_name = 'tokens'):
	print(f"Before: {df.shape}")

	# Remove duplicate data
	df.drop_duplicates(subset=key_name, keep="first", inplace=True)

	# Remove empty data
	df.dropna(subset=[key_name], inplace=True)

	# Remove data with length less than 2
	df = df[df[key_name].map(len) > 2]
	df.reset_index(inplace=True)

	print(f"After: {df.shape}")
	return df

# threshold is 10% of the total data
threshold = int(df.shape[0] * 0.1)
df = remove_data_with_rare_word(df, key_name='tokens', threshold=threshold)
df = remove_data(df, key_name='tokens')

df

Before: (7502, 6)
After: (6864, 6)
Before: (6864, 6)
After: (6485, 7)


,index,date,content,text_clean,case_folding,tokens,review
0,0,2022-09-04 23:59:55+00:00,@soeyoto1 @msaid_didu Bukti kalau Pemerintahan Pak @jokowi tidak pro rakyat. Tapi pro oligarki. ...,Bukti kalau Pemerintahan Pak tidak pro rakyat Tapi pro oligarki Vivo BBMnya lebih murah dari Pem...,bukti kalau pemerintahan pak tidak pro rakyat tapi pro oligarki vivo bbmnya lebih murah dari pem...,"[bukti, kalau, pemerintahan, tidak, pro, rakyat, pro, oligarki, vivo, bbmnya, lebih, murah, dari...",bukti kalau pemerintahan tidak pro rakyat pro oligarki vivo bbmnya lebih murah dari pemerintah n...
1,1,2022-09-04 23:59:38+00:00,"@KompasTV Mentri esdm, trus patokanmu apa? Kantong pertamina udah jebol karna banyak dikorupsi, ...",Mentri esdm trus patokanmu apa Kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...,mentri esdm trus patokanmu apa kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...,"[mentri, esdm, trus, patokanmu, apa, kantong, pertamina, udah, jebol, karna, banyak, dikorupsi, ...",mentri esdm trus patokanmu apa kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...
2,2,2022-09-04 23:59:09+00:00,@hnurwahid Ngaku masih subsidi tapi ada swasta yg jua lebih murah? mereka siapa yg subsidi? Pert...,Ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi Pertamina dipimpin tuka...,ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi pertamina dipimpin tuka...,"[ngaku, subsidi, ada, swasta, jua, lebih, murah, siapa, subsidi, pertamina, dipimpin, tukang, ma...",ngaku subsidi ada swasta jua lebih murah siapa subsidi pertamina dipimpin tukang maki jadi begin...
3,3,2022-09-04 23:58:55+00:00,@Tan_Mar3M Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina ? 🤔,Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina,kalau vivo bisaberani bertahan dengan harga lama bagaimana pula nasib pertamina,"[kalau, vivo, bisaberani, bertahan, harga, lama, nasib, pertamina]",kalau vivo bisaberani bertahan harga lama nasib pertamina
4,4,2022-09-04 23:57:45+00:00,Menteri BUMN Erick Thohir menegaskan persiapan Indonesia soal transisi energi baru terbarukan. S...,Menteri BUMNErick Thohirmenegaskan persiapan Indonesia soal transisi energi baru terbarukan Sala...,menteri bumnerick thohirmenegaskan persiapan indonesia soal transisi energi baru terbarukan sala...,"[menteri, bumnerick, thohirmenegaskan, persiapan, indonesia, soal, transisi, energi, terbarukan,...",menteri bumnerick thohirmenegaskan persiapan indonesia soal transisi energi terbarukan salah sat...
...,...,...,...,...,...,...,...
6480,6859,2022-09-03 00:00:54+00:00,"175 Produk UMKM Binaan Pertamina Go Global, Ikut Festival Tong Tong di Belanda https://t.co/LTf7...",Produk UMKM Binaan Pertamina Global Ikut Festival Tong Tong Belanda,produk umkm binaan pertamina global ikut festival tong tong belanda,"[produk, umkm, binaan, pertamina, global, ikut, festival, tong, tong, belanda]",produk umkm binaan pertamina global ikut festival tong tong belanda
6481,6860,2022-09-03 00:00:36+00:00,"PT Pertamina Patra Niaga, Sub Holding Commercial dan Trading PT Pertamina (Persero) menyebutkan ...",Pertamina Patra Niaga Sub Holding Commercial dan Trading Pertamina Persero menyebutkan telah mel...,pertamina patra niaga sub holding commercial dan trading pertamina persero menyebutkan telah mel...,"[pertamina, patra, niaga, sub, holding, commercial, trading, pertamina, persero, penyesuaian, ha...",pertamina patra niaga sub holding commercial trading pertamina persero penyesuaian harga bbm non...
6482,6861,2022-09-03 00:00:02+00:00,Pemerintah Provinsi Papua Barat sedang mengupayakan dengan mengajak Pertamina agar masalah antre...,Pemerintah Provinsi Papua Barat sedang mengupayakan dengan mengajak Pertamina agar masalah antre...,pemerintah provinsi papua barat sedang mengupayakan dengan mengajak pertamina agar masalah antre...,"[pemerintah, provinsi, papua, barat, mengupayakan, mengajak, pertami

In [9]:
df['tokens'] = df['tokens'].swifter.apply(remove_repetitive_char)

Pandas Apply:   0%|          | 0/6485 [00:00<?, ?it/s]

# Section 1: Data Stemming

In [10]:
terms_dict = stemming(df['tokens'])

Unique terms: 12582
On processing... 0 terms have been stemmed
On processing... 1000 terms have been stemmed
On processing... 2000 terms have been stemmed
On processing... 3000 terms have been stemmed
On processing... 4000 terms have been stemmed
On processing... 5000 terms have been stemmed
On processing... 6000 terms have been stemmed
On processing... 7000 terms have been stemmed
On processing... 8000 terms have been stemmed
On processing... 9000 terms have been stemmed
On processing... 10000 terms have been stemmed
On processing... 11000 terms have been stemmed
On processing... 12000 terms have been stemmed


In [11]:
def apply_stemmed_tokens(tokens):
	return [terms_dict[token] for token in tokens]

df['tokens'] = df['tokens'].swifter.apply(apply_stemmed_tokens)
df['review'] = df['tokens'].swifter.apply(combine_tokens)

print(df.shape)
df

Pandas Apply:   0%|          | 0/6485 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/6485 [00:00<?, ?it/s]

(6485, 7)


,index,date,content,text_clean,case_folding,tokens,review
0,0,2022-09-04 23:59:55+00:00,@soeyoto1 @msaid_didu Bukti kalau Pemerintahan Pak @jokowi tidak pro rakyat. Tapi pro oligarki. ...,Bukti kalau Pemerintahan Pak tidak pro rakyat Tapi pro oligarki Vivo BBMnya lebih murah dari Pem...,bukti kalau pemerintahan pak tidak pro rakyat tapi pro oligarki vivo bbmnya lebih murah dari pem...,"[bukti, kalau, perintah, tidak, pro, rakyat, pro, oligarki, vivo, bmnya, lebih, murah, dari, per...",bukti kalau perintah tidak pro rakyat pro oligarki vivo bmnya lebih murah dari perintah naikin a...
1,1,2022-09-04 23:59:38+00:00,"@KompasTV Mentri esdm, trus patokanmu apa? Kantong pertamina udah jebol karna banyak dikorupsi, ...",Mentri esdm trus patokanmu apa Kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...,mentri esdm trus patokanmu apa kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...,"[tri, esdm, trus, patok, apa, kantong, pertamina, udah, jebol, karna, banyak, korupsi, imbas, ra...",tri esdm trus patok apa kantong pertamina udah jebol karna banyak korupsi imbas rakyat yang mena...
2,2,2022-09-04 23:59:09+00:00,@hnurwahid Ngaku masih subsidi tapi ada swasta yg jua lebih murah? mereka siapa yg subsidi? Pert...,Ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi Pertamina dipimpin tuka...,ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi pertamina dipimpin tuka...,"[ngaku, subsidi, ada, swasta, jua, lebih, murah, siapa, subsidi, pertamina, pimpin, tukang, maki...",ngaku subsidi ada swasta jua lebih murah siapa subsidi pertamina pimpin tukang maki jadi begini ...
3,3,2022-09-04 23:58:55+00:00,@Tan_Mar3M Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina ? 🤔,Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina,kalau vivo bisaberani bertahan dengan harga lama bagaimana pula nasib pertamina,"[kalau, vivo, bisaberani, tahan, harga, lama, nasib, pertamina]",kalau vivo bisaberani tahan harga lama nasib pertamina
4,4,2022-09-04 23:57:45+00:00,Menteri BUMN Erick Thohir menegaskan persiapan Indonesia soal transisi energi baru terbarukan. S...,Menteri BUMNErick Thohirmenegaskan persiapan Indonesia soal transisi energi baru terbarukan Sala...,menteri bumnerick thohirmenegaskan persiapan indonesia soal transisi energi baru terbarukan sala...,"[menteri, bumnerick, thohirmenegaskan, siap, indonesia, soal, transisi, energi, baru, salah, sat...",menteri bumnerick thohirmenegaskan siap indonesia soal transisi energi baru salah satu upaya lew...
...,...,...,...,...,...,...,...
6480,6859,2022-09-03 00:00:54+00:00,"175 Produk UMKM Binaan Pertamina Go Global, Ikut Festival Tong Tong di Belanda https://t.co/LTf7...",Produk UMKM Binaan Pertamina Global Ikut Festival Tong Tong Belanda,produk umkm binaan pertamina global ikut festival tong tong belanda,"[produk, umkm, bin, pertamina, global, ikut, festival, tong, tong, belanda]",produk umkm bin pertamina global ikut festival tong tong belanda
6481,6860,2022-09-03 00:00:36+00:00,"PT Pertamina Patra Niaga, Sub Holding Commercial dan Trading PT Pertamina (Persero) menyebutkan ...",Pertamina Patra Niaga Sub Holding Commercial dan Trading Pertamina Persero menyebutkan telah mel...,pertamina patra niaga sub holding commercial dan trading pertamina persero menyebutkan telah mel...,"[pertamina, patra, niaga, sub, holding, comercial, trading, pertamina, persero, sesuai, harga, b...",pertamina patra niaga sub holding comercial trading pertamina persero sesuai harga bm non subsid...
6482,6861,2022-09-03 00:00:02+00:00,Pemerintah Provinsi Papua Barat sedang mengupayakan dengan mengajak Pertamina agar masalah antre...,Pemerintah Provinsi Papua Barat sedang mengupayakan dengan mengajak Pertamina agar masalah antre...,pemerintah provinsi papua barat sedang mengupayakan dengan mengajak pertamina agar masalah antre...,"[perintah, provinsi, papua, barat, upaya, ajak, pertamina, agar, masalah, antre,

In [12]:
duplicate_data = df[df.duplicated(subset='review')].sort_values(by='review')

duplicate_data

,index,date,content,text_clean,case_folding,tokens,review
1575,1638,2022-09-04 09:58:28+00:00,PASTIKAN STOK BBM AMAN DIRUT PERTAMINA PANTAU LANGSUNG COMAND CENTER,PASTIKAN STOK BBM AMAN DIRUT PERTAMINA PANTAU LANGSUNG COMAND CENTER,pastikan stok bbm aman dirut pertamina pantau langsung comand center,"[pasti, stok, bm, aman, dirut, pertamina, pantau, langsung, comand, center]",pasti stok bm aman dirut pertamina pantau langsung comand center
5625,5897,2022-09-03 10:01:19+00:00,"Pertamina pastikan ketersediaan stok BBM subsidi untuk masyarakat 𝘽𝘽𝙈, 𝙎𝙋𝘽𝙐, 𝙋𝙚𝙧𝙩𝙖𝙢𝙞𝙣𝙖 𝘼𝙢𝙖𝙣 http...",Pertamina pastikan ketersediaan stok BBM subsidi untuk masyarakat,pertamina pastikan ketersediaan stok bbm subsidi untuk masyarakat,"[pertamina, pasti, dian, stok, bm, subsidi, masyarakat]",pertamina pasti dian stok bm subsidi masyarakat
5619,5891,2022-09-03 10:02:06+00:00,"PIEDCC kunci Pertamina dalam menjaga ketersediaan stok BBM subsidi 𝘽𝘽𝙈, 𝙎𝙋𝘽𝙐, 𝙋𝙚𝙧𝙩𝙖𝙢𝙞𝙣𝙖 𝘼𝙢𝙖𝙣 htt...",PIEDCC kunci Pertamina dalam menjaga ketersediaan stok BBM subsidi,piedcc kunci pertamina dalam menjaga ketersediaan stok bbm subsidi,"[piedc, kunci, pertamina, dalam, jaga, dian, stok, bm, subsidi]",piedc kunci pertamina dalam jaga dian stok bm subsidi


In [13]:
df['tokens'] = df['tokens'].swifter.apply(remove_stopwords, args=(list_stopwords,))
df['review'] = df['tokens'].swifter.apply(combine_tokens)

df = remove_data(df, 'review')

print(df.shape)
df.head()

Pandas Apply:   0%|          | 0/6485 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/6485 [00:00<?, ?it/s]

Before: (6485, 7)
After: (6479, 8)
(6479, 8)


,level_0,index,date,content,text_clean,case_folding,tokens,review
0,0,0,2022-09-04 23:59:55+00:00,@soeyoto1 @msaid_didu Bukti kalau Pemerintahan Pak @jokowi tidak pro rakyat. Tapi pro oligarki. ...,Bukti kalau Pemerintahan Pak tidak pro rakyat Tapi pro oligarki Vivo BBMnya lebih murah dari Pem...,bukti kalau pemerintahan pak tidak pro rakyat tapi pro oligarki vivo bbmnya lebih murah dari pem...,"[bukti, kalau, perintah, tidak, pro, rakyat, pro, oligarki, vivo, bmnya, lebih, murah, dari, per...",bukti kalau perintah tidak pro rakyat pro oligarki vivo bmnya lebih murah dari perintah naikin a...
1,1,1,2022-09-04 23:59:38+00:00,"@KompasTV Mentri esdm, trus patokanmu apa? Kantong pertamina udah jebol karna banyak dikorupsi, ...",Mentri esdm trus patokanmu apa Kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...,mentri esdm trus patokanmu apa kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...,"[tri, esdm, trus, patok, apa, kantong, pertamina, udah, jebol, karna, banyak, korupsi, imbas, ra...",tri esdm trus patok apa kantong pertamina udah jebol karna banyak korupsi imbas rakyat yang mena...
2,2,2,2022-09-04 23:59:09+00:00,@hnurwahid Ngaku masih subsidi tapi ada swasta yg jua lebih murah? mereka siapa yg subsidi? Pert...,Ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi Pertamina dipimpin tuka...,ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi pertamina dipimpin tuka...,"[ngaku, subsidi, ada, swasta, jua, lebih, murah, siapa, subsidi, pertamina, pimpin, tukang, maki...",ngaku subsidi ada swasta jua lebih murah siapa subsidi pertamina pimpin tukang maki jadi begini ...
3,3,3,2022-09-04 23:58:55+00:00,@Tan_Mar3M Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina ? 🤔,Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina,kalau vivo bisaberani bertahan dengan harga lama bagaimana pula nasib pertamina,"[kalau, vivo, bisaberani, tahan, harga, lama, nasib, pertamina]",kalau vivo bisaberani tahan harga lama nasib pertamina
4,4,4,2022-09-04 23:57:45+00:00,Menteri BUMN Erick Thohir menegaskan persiapan Indonesia soal transisi energi baru terbarukan. S...,Menteri BUMNErick Thohirmenegaskan persiapan Indonesia soal transisi energi baru terbarukan Sala...,menteri bumnerick thohirmenegaskan persiapan indonesia soal transisi energi baru terbarukan sala...,"[menteri, bumnerick, thohirmenegaskan, siap, indonesia, soal, transisi, energi, salah, satu, upa...",menteri bumnerick thohirmenegaskan siap indonesia soal transisi energi salah satu upaya lewat ke...


In [14]:
df

,level_0,index,date,content,text_clean,case_folding,tokens,review
0,0,0,2022-09-04 23:59:55+00:00,@soeyoto1 @msaid_didu Bukti kalau Pemerintahan Pak @jokowi tidak pro rakyat. Tapi pro oligarki. ...,Bukti kalau Pemerintahan Pak tidak pro rakyat Tapi pro oligarki Vivo BBMnya lebih murah dari Pem...,bukti kalau pemerintahan pak tidak pro rakyat tapi pro oligarki vivo bbmnya lebih murah dari pem...,"[bukti, kalau, perintah, tidak, pro, rakyat, pro, oligarki, vivo, bmnya, lebih, murah, dari, per...",bukti kalau perintah tidak pro rakyat pro oligarki vivo bmnya lebih murah dari perintah naikin a...
1,1,1,2022-09-04 23:59:38+00:00,"@KompasTV Mentri esdm, trus patokanmu apa? Kantong pertamina udah jebol karna banyak dikorupsi, ...",Mentri esdm trus patokanmu apa Kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...,mentri esdm trus patokanmu apa kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...,"[tri, esdm, trus, patok, apa, kantong, pertamina, udah, jebol, karna, banyak, korupsi, imbas, ra...",tri esdm trus patok apa kantong pertamina udah jebol karna banyak korupsi imbas rakyat yang mena...
2,2,2,2022-09-04 23:59:09+00:00,@hnurwahid Ngaku masih subsidi tapi ada swasta yg jua lebih murah? mereka siapa yg subsidi? Pert...,Ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi Pertamina dipimpin tuka...,ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi pertamina dipimpin tuka...,"[ngaku, subsidi, ada, swasta, jua, lebih, murah, siapa, subsidi, pertamina, pimpin, tukang, maki...",ngaku subsidi ada swasta jua lebih murah siapa subsidi pertamina pimpin tukang maki jadi begini ...
3,3,3,2022-09-04 23:58:55+00:00,@Tan_Mar3M Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina ? 🤔,Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina,kalau vivo bisaberani bertahan dengan harga lama bagaimana pula nasib pertamina,"[kalau, vivo, bisaberani, tahan, harga, lama, nasib, pertamina]",kalau vivo bisaberani tahan harga lama nasib pertamina
4,4,4,2022-09-04 23:57:45+00:00,Menteri BUMN Erick Thohir menegaskan persiapan Indonesia soal transisi energi baru terbarukan. S...,Menteri BUMNErick Thohirmenegaskan persiapan Indonesia soal transisi energi baru terbarukan Sala...,menteri bumnerick thohirmenegaskan persiapan indonesia soal transisi energi baru terbarukan sala...,"[menteri, bumnerick, thohirmenegaskan, siap, indonesia, soal, transisi, energi, salah, satu, upa...",menteri bumnerick thohirmenegaskan siap indonesia soal transisi energi salah satu upaya lewat ke...
...,...,...,...,...,...,...,...,...
6474,6480,6859,2022-09-03 00:00:54+00:00,"175 Produk UMKM Binaan Pertamina Go Global, Ikut Festival Tong Tong di Belanda https://t.co/LTf7...",Produk UMKM Binaan Pertamina Global Ikut Festival Tong Tong Belanda,produk umkm binaan pertamina global ikut festival tong tong belanda,"[produk, umkm, bin, pertamina, global, ikut, festival, tong, tong, belanda]",produk umkm bin pertamina global ikut festival tong tong belanda
6475,6481,6860,2022-09-03 00:00:36+00:00,"PT Pertamina Patra Niaga, Sub Holding Commercial dan Trading PT Pertamina (Persero) menyebutkan ...",Pertamina Patra Niaga Sub Holding Commercial dan Trading Pertamina Persero menyebutkan telah mel...,pertamina patra niaga sub holding commercial dan trading pertamina persero menyebutkan telah mel...,"[pertamina, patra, niaga, sub, holding, comercial, trading, pertamina, persero, sesuai, harga, b...",pertamina patra niaga sub holding comercial trading pertamina persero sesuai harga bm non subsid...
6476,6482,6861,2022-09-03 00:00:02+00:00,Pemerintah Provinsi Papua Barat sedang mengupayakan dengan mengajak Pertamina agar masalah antre...,Pemerintah Provinsi Papua Barat sedang mengupayakan dengan mengajak Pertamina agar masalah antre...,pemerintah provinsi papua barat sedang mengupayakan dengan mengajak pertamina agar masalah antre...,"[perintah, provinsi, papua, barat, upaya, a

In [15]:
df.to_csv(output_path, index=False, header=True, sep=';')